# Monte Carlo Sampling

Compare with:

+ Arc
+ Act 1 & 2
+ Demo (Pulsed and SS)

// also adds prototype reactor designs (proteus and charybdis)

## 0.0 Bootup Notebook

In [1]:
# if Sys.CPU_CORES != nprocs()
#     rmprocs(collect(2:100))
#     addprocs(Sys.CPU_CORES-1)
# end

# addprocs(12)

# @everywhere 

using Fussy
Main.IJulia.set_current_module(Fussy)

return

## 1.0 Set Defaults

In [2]:
cur_decks = [
    :proteus, # pulsed
    :charybdis, # steady state
    :arc, 
    :act_1, 
    :act_2,
    :demo_steady, 
    :demo_pulsed
]

cur_sensitivity = 0.35

cur_params = [ 
    :H, :Q, :wave_theta,
    :epsilon, :delta_95,
    :nu_n, :nu_T, :l_i,
    :N_G, :f_D, :Z_eff,
    :eta_CD, :B_CS, :tau_FT
]

return

## 2.0 Define Make Function

In [3]:
gr()

LoadError: [91mUndefVarError: gr not defined[39m

In [21]:
import Interact: @manipulate
using FileIO
using JLD2
using Plots
using StringCases

In [5]:
function make_sampling(cur_deck)
    if cur_deck == :demo_pulsed || cur_deck == :proteus || cur_deck == :demo_steady
        is_consistent = false
    else
        is_consistent = true
    end
    
    cur_sampling = Sampling(cur_params; deck=cur_deck, is_consistent=is_consistent, study_count=50)
    
    cur_sampling
end

return

## 3.0 Get Data

In [6]:
cur_samplings = nothing 

try
    cur_samplings = load("../data/samplings.jld2", "cur_samplings") 
catch
    cur_samplings = Dict()
end

for cur_deck in cur_decks
    haskey(cur_samplings, cur_deck) && continue
    println(cur_deck)
    
    cur_samplings[cur_deck] = make_sampling(cur_deck)
    
    jldopen("../data/samplings.jld2", "w") do work_dict
        work_dict["cur_samplings"] = cur_samplings
    end
end

for (cur_key, cur_value) in cur_samplings
    for cur_index in 1:cur_value.study_count*100
        try
            cur_value.wall_reactors[cur_index]
        catch cur_error
            println(cur_index, " - ", ( cur_index == cur_value.study_count + 1 ))
            break 
        end
        if is_present(cur_value.wall_reactors[cur_index]) && is_present(cur_value.cost_reactors[cur_index])
            cur_error = abs(cur_value.wall_reactors[cur_index].cost-cur_value.cost_reactors[cur_index].cost)
            ( cur_error < 1e-4 ) && ( cur_value.cost_reactors[cur_index] = nothing )
        end
        if is_present(cur_value.cost_reactors[cur_index]) && is_present(cur_value.W_M_reactors[cur_index])
            cur_error = abs(cur_value.cost_reactors[cur_index].cost-cur_value.W_M_reactors[cur_index].cost)
            ( cur_error < 1e-4 ) && ( cur_value.cost_reactors[cur_index] = nothing )
        end
        if is_present(cur_value.kink_reactors[cur_index]) && is_present(cur_value.W_M_reactors[cur_index])
            cur_error = abs(cur_value.kink_reactors[cur_index].cost-cur_value.W_M_reactors[cur_index].cost)
            ( cur_error < 1e-4 ) && ( cur_value.W_M_reactors[cur_index] = nothing )
        end
        if is_present(cur_value.kink_reactors[cur_index]) && is_present(cur_value.cost_reactors[cur_index])
            cur_error = abs(cur_value.kink_reactors[cur_index].cost-cur_value.cost_reactors[cur_index].cost)
            ( cur_error < 1e-4 ) && ( cur_value.cost_reactors[cur_index] = nothing )
        end
        if is_present(cur_value.wall_reactors[cur_index]) && is_present(cur_value.W_M_reactors[cur_index])
            cur_error = abs(cur_value.wall_reactors[cur_index].cost-cur_value.W_M_reactors[cur_index].cost)
            ( cur_error < 1e-4 ) && ( cur_value.W_M_reactors[cur_index] = nothing )
        end
        if is_present(cur_value.wall_reactors[cur_index]) && is_present(cur_value.kink_reactors[cur_index])
            cur_error = abs(cur_value.wall_reactors[cur_index].cost-cur_value.kink_reactors[cur_index].cost)
            ( cur_error < 1e-4 ) && ( cur_value.wall_reactors[cur_index] = nothing )
        end
    end
end

println("done.")

# return

1301 - true
701 - true
1051 - true
1051 - true
951 - true
1301 - true
1051 - true
done.


## 4.0 Setup for Plots

In [7]:
cur_fields = [ 
    "T_bar", "n_bar", "I_P", "R_0", "B_0", "R_CS",
    "P_F", "f_IN", "f_BS", "f_CD", "W_M", "cost", "eta_CD",
    "norm_beta_N", "norm_q_95", "norm_P_E", "norm_P_W", "b", "c", "d", "P_E"
]

cur_fields = map(zz -> Symbol(zz), cur_fields)

append!(cur_fields, cur_params)

cur_fields = sort(cur_fields)

cur_x_list = deepcopy(cur_fields)
cur_y_list = deepcopy(cur_fields)

deleteat!(cur_x_list, find(cur_field -> cur_field == :B_0, cur_x_list))
deleteat!(cur_y_list, find(cur_field -> cur_field == :cost, cur_y_list))

deleteat!(cur_x_list, find(cur_field -> cur_field == :cost, cur_x_list))
deleteat!(cur_y_list, find(cur_field -> cur_field == :B_0, cur_y_list))

unshift!(cur_y_list, :B_0)
unshift!(cur_x_list, :cost)

unshift!(cur_x_list, :B_0)
unshift!(cur_y_list, :cost)


35-element Array{Symbol,1}:
 :cost       
 :B_0        
 :B_CS       
 :H          
 :I_P        
 :N_G        
 :P_E        
 :P_F        
 :Q          
 :R_0        
 :R_CS       
 :T_bar      
 :W_M        
 ⋮           
 :f_D        
 :f_IN       
 :l_i        
 :n_bar      
 :norm_P_E   
 :norm_P_W   
 :norm_beta_N
 :norm_q_95  
 :nu_T       
 :nu_n       
 :tau_FT     
 :wave_theta 

## 5.0 Make Plot GUI

In [1]:
markers = [:diamond, :square,:circle, :pentagon]
limits = true
constraints = true

@manipulate for y in cur_y_list, x in cur_x_list, z in [:W_M, :P_F, :cost], w in [:P_F, :W_M, :cost], yscale=[:log, :lin], xscale=[:lin, :log], deck in cur_decks, simple=[true,false], legend=[true,false]#, limits=true,constraints=true
    plot()
    
    cur_list = [:kink, :wall, :cost, :W_M]
    
    this_xx = []
    this_yy = []
    this_zz = []
    this_ww = []
    this_ss = []
    this_mm = []
    this_oo = []
    
    for (tmp_index, kind) in enumerate(cur_list)
        tmp_reacs = deepcopy(getfield(cur_samplings[deck], Symbol("$(kind)_reactors")))
        filter!(is_present, tmp_reacs)
        filter!(tmp_reac -> tmp_reac.is_valid, tmp_reacs)
        filter!(tmp_reac -> tmp_reac.is_good, tmp_reacs)

        filter!(tmp_reac -> tmp_reac.B_0 < 25, tmp_reacs)
        filter!(tmp_reac -> tmp_reac.R_0 < 25, tmp_reacs)
        filter!(tmp_reac -> tmp_reac.cost < 0.1, tmp_reacs)
        
        filter!(tmp_reac -> tmp_reac.norm_P_E < 0.8, tmp_reacs)
        
        isempty(tmp_reacs) && continue
        
        cur_xx = map(tmp_reac -> getfield(tmp_reac, x), tmp_reacs)
        cur_yy = map(tmp_reac -> getfield(tmp_reac, y), tmp_reacs)
        cur_zz = map(tmp_reac -> getfield(tmp_reac, z), tmp_reacs)
        cur_ww = map(tmp_reac -> getfield(tmp_reac, w), tmp_reacs)
        
        cur_mm = []
        for tmp_reac in tmp_reacs
            if simple 
                if isapprox(tmp_reac.norm_beta_N,1.0,rtol=1e-4)
                    tmp_m = markers[3] #:beta
                elseif isapprox(tmp_reac.norm_P_W,1.0,rtol=1e-4)
                    tmp_m = markers[2] #:wall
                else
                    @assert isapprox(tmp_reac.norm_q_95,1.0,rtol=1e-4)
                    tmp_m = markers[1] #:kink
                end
            else
                tmp_m = markers[tmp_index]
            end
            push!(cur_mm, tmp_m)
        end
    
        append!(this_oo, (tmp_index < 3 ? Int(limits) : Int(constraints))*ones(length(cur_zz)))
        append!(this_mm, cur_mm)
        append!(this_ss, map(tmp_m -> tmp_m ==:square ? 4/sqrt(2) : 4, cur_mm))
#         cur_ww = map(tmp_reac -> log10(getfield(tmp_reac, w)), tmp_reacs)
#         cur_ww .-= minimum(cur_ww)
#         cur_ww ./= maximum(cur_ww)
#         cur_ww .*= -1p
#         cur_ww .+= 1

#         cur_ww .-= 0.5
#         cur_ww *= 2

        append!(this_xx, cur_xx)
        append!(this_zz, cur_zz)
        append!(this_yy, cur_yy)
        append!(this_ww, cur_ww)
    end
    
    max_x = maximum(this_xx)
    max_y = maximum(this_yy)
    
    min_x = minimum(this_xx)
    min_y = minimum(this_yy)

    this_ww = -log10.(this_ww)
    
    this_ww -= minimum(this_ww)
    this_ww /= maximum(this_ww)
    
    this_ss += 2 * this_ww - 1
    
#     this_ss += 2
    
    this_zz = -log10.(this_zz)
    println(length(this_zz))
    
    sort_lists!(this_zz,this_xx,this_yy,this_ss,this_mm,this_oo,this_ww)

    this_oo .*= (ceil.((0.6+0.4*this_ww)*10)/10)
    this_ss = map(round,this_ss)
    
    plot!([],[],label="color = -log( $z )", opacity=0)
    plot!([],[],label="size = a - b * log( $w )", opacity=0)
    plot!([],[],label=" ", opacity=0)
    
    for (tmp_index, tmp_label) in enumerate(cur_list)
        cur_label = nothing
        if simple
            ( tmp_index == 4 ) && continue
            
            cur_label = tmp_index == 3 ? :beta : tmp_label
        else
            
            cur_label = tmp_index < 3 ? "$(tmp_label)-beta" : tmp_label
        end
        scatter!([],[],color=tmp_index < 3 ? tmp_index : tmp_index < 4  ? tmp_index + 1 : tmp_index + 3,label=cur_label,marker=markers[tmp_index])
    end
    scatter!(this_xx, this_yy,zcolor=this_zz, color=:viridis,markersize=this_ss, markershapes=this_mm, label="",opacity=this_oo, colorbar_title="Asdf")#, markersize=marker_size),marker=markers[tmp_index], label=kind

    max_x *= 1.25
    max_y *= 1.25
    
    in(x, [:R_0, :B_0]) && ( max_x = 25 )
    in(y, [:R_0, :B_0]) && ( max_y = 25 )
    
    in(x, [:f_IN, :f_BS, :f_CD, :norm_beta_N, :norm_P_W, :norm_q_95]) && ( max_x = 1.25 )
    in(y, [:f_IN, :f_BS, :f_CD, :norm_beta_N, :norm_P_W, :norm_q_95]) && ( max_y = 1.25 )
    
    ( x == :cost ) && ( max_x = 0.1 )
    ( y == :cost ) && ( max_y = 0.1 )
    
    min_x = xscale == :lin ? 0 : min_x / 1.25
    min_y = yscale == :lin ? 0 : min_y / 1.25
   
    ( x == :cost ) && ( min_x = min(min_x, 0.001) )
    ( y == :cost ) && ( min_y = min(min_y, 0.001) )
    
    xlims!(min_x, max_x)
    ylims!(min_y, max_y)
    
    ( yscale == :lin ) || plot!(yscale = yscale)
    ( xscale == :lin ) || plot!(xscale = xscale)
    
    ylabel!(string(y))
    xlabel!(string(x))
    
    plot!(dpi=450)

    cur_title = string(deck)
    cur_title = join(map(capitalize,split(cur_title, "_")), " ")
    
    title!(cur_title)
    plot!(legend=legend)
end

LoadError: [91mUndefVarError: @manipulate not defined[39m